In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dr="/content/drive/My Drive/Classroom/Datasets/"

In [71]:
import os
os.listdir(dr)

['glove.6B',
 'donar_preprocess.csv',
 'donar_tfselected_words.csv',
 'chunk3_train_sparse_matrix.npz',
 'chunk2_train_sparse_matrix.npz',
 'chunk1_train_sparse_matrix.npz',
 'sample_test_sparse_matrix.npz',
 'Donesaving_reg1.csv',
 'reg_test.csv',
 'reg_train1.csv',
 'Donesaving_reg2.csv',
 'reg_train2.csv',
 'reg_train3.csv',
 'random_file_size.csv',
 'srandom_parse_matrix.npz',
 'top_feat_names.npy',
 'top_feat_indexs.npy',
 'file_name_list.npy',
 'file_size.csv',
 'samplebyte_images.rar',
 'samplebyte_images',
 'trainLabels.csv',
 'result.csv',
 'asmoutputfile.csv',
 'asm_result_with_size.csv',
 'one_data.csv',
 'final_byte.csv',
 'cnn_feat.csv',
 'IMDB_files_csv',
 'en_train.csv',
 'model_INT',
 'label_model.h5']

In [0]:
# this is just to know how much time will it take to run this entire ipython notebook 
from datetime import datetime
# globalstart = datetime.now()
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

In [5]:
import numpy as np
import string
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector, Activation, TimeDistributed, Dense, RepeatVector, Embedding, Bidirectional,Flatten
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
import warnings
from keras.callbacks import ModelCheckpoint
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
df=pd.read_csv(dr+"en_train.csv")

In [0]:
data=df[df['sentence_id']<=150000]#taking 1.5 million sentence data

In [9]:
data['class'].value_counts()

PLAIN         1418821
PUNCT          367436
DATE            54281
LETTERS         28991
CARDINAL        25820
VERBATIM        15076
MEASURE          3092
ORDINAL          2385
DECIMAL          2022
MONEY            1224
DIGIT            1011
ELECTRONIC        957
TELEPHONE         738
TIME              245
FRACTION          228
ADDRESS            96
Name: class, dtype: int64

In [0]:
data["new_class"]=data['class'].apply(lambda c: "NUMERIC" if str(c)=='DATE' or  str(c)=='CARDINAL' or  str(c)=='MEASURE' or  str(c)=='ORDINAL' or  str(c)=='DECIMAL' or  str(c)=='MONEY' or  str(c)=='DIGIT' or  str(c)=='TELEPHONE' or str(c)=='TIME' or str(c)=='FRACTION' or  str(c)=='ADDRESS' else( "PLAIN_PUNC" if str(c)=='PLAIN' or str(c)=='PUNCT' else( "LETTERS_VERB" if str(c)=='LETTERS' or str(c)=='VERBATIM' else c ) ))

In [11]:
data["new_class"].value_counts()

PLAIN_PUNC      1786257
NUMERIC           91142
LETTERS_VERB      44067
ELECTRONIC          957
Name: new_class, dtype: int64

In [30]:
data.shape

(1922423, 6)

In [13]:
data.head(11)

,sentence_id,token_id,class,before,after,new_class
0,0,0,PLAIN,Brillantaisia,Brillantaisia,PLAIN_PUNC
1,0,1,PLAIN,is,is,PLAIN_PUNC
2,0,2,PLAIN,a,a,PLAIN_PUNC
3,0,3,PLAIN,genus,genus,PLAIN_PUNC
4,0,4,PLAIN,of,of,PLAIN_PUNC
5,0,5,PLAIN,plant,plant,PLAIN_PUNC
6,0,6,PLAIN,in,in,PLAIN_PUNC
7,0,7,PLAIN,family,family,PLAIN_PUNC
8,0,8,PLAIN,Acanthaceae,Acanthaceae,PLAIN_PUNC
9,0,9,PUNCT,.,.,PLAIN_PUNC


In [0]:
df=data[data['new_class']=='LETTERS_VERB']
num_df=data[data['new_class']=='NUMERIC']
df.columns=['sentence_id',	'token_id',	'class',	'written',	'spoken',	'new_class']

In [54]:
df.head()

,sentence_id,token_id,class,written,spoken,new_class
11,1,1,LETTERS,IUCN,i u c n,LETTERS_VERB
115,9,2,LETTERS,BC,b c,LETTERS_VERB
159,14,2,LETTERS,ALCS,a l c s,LETTERS_VERB
162,14,5,VERBATIM,-,-,LETTERS_VERB
201,18,2,LETTERS,C.,c,LETTERS_VERB


In [0]:

def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
#tokenizer.texts_to_sequences('str')

In [0]:
df[['spoken','written']]=df[['spoken','written']].astype('str')

In [57]:
# prepare  spoken tokenizer

#spoken_tokenizer =Tokenizer()
spoken_tokenizer=tokenization(df["spoken"])
spoken_vocab_size = len(spoken_tokenizer.word_index) + 1

spoken_length = 20
print('Size of English spoken Vocabulary : %d' % spoken_vocab_size)

Size of English spoken Vocabulary : 1376


In [58]:
# prepare written tokenizer
#written_tokenizer =Tokenizer()
written_tokenizer=tokenization(df["written"])
written_vocab_size = len(written_tokenizer.word_index) + 1

written_length = 20
print('Size of English written Vocabulary : %d' % written_vocab_size)

Size of English written Vocabulary : 7723


In [0]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

## Model building

In [60]:
df['spoken'].apply(lambda l: len(l))

11         7
115        3
159        7
162        1
201        1
          ..
1921948    3
1921983    3
1922171    3
1922247    1
1922320    3
Name: spoken, Length: 44067, dtype: int64

In [61]:
df.head(15)

,sentence_id,token_id,class,written,spoken,new_class
11,1,1,LETTERS,IUCN,i u c n,LETTERS_VERB
115,9,2,LETTERS,BC,b c,LETTERS_VERB
159,14,2,LETTERS,ALCS,a l c s,LETTERS_VERB
162,14,5,VERBATIM,-,-,LETTERS_VERB
201,18,2,LETTERS,C.,c,LETTERS_VERB
205,18,6,LETTERS,J.,j,LETTERS_VERB
209,18,10,LETTERS,G.,g,LETTERS_VERB
224,19,1,LETTERS,U.S.,u s,LETTERS_VERB
279,24,3,VERBATIM,#,number,LETTERS_VERB
317,26,8,LETTERS,LP,l p,LETTERS_VERB


In [0]:
#SPLITTING DATA
from sklearn.model_selection import train_test_split
train, test = train_test_split(df[['spoken','written']], test_size=0.28, random_state = 42)
# prepare training data
trainX = encode_sequences(spoken_tokenizer, spoken_length, train.iloc[:,0])
trainY = encode_sequences(written_tokenizer, written_length, train.iloc[:,1])
# prepare validation data
testX = encode_sequences(spoken_tokenizer, spoken_length, test.iloc[:,0])
testY = encode_sequences(written_tokenizer, written_length, test.iloc[:,0])

In [63]:
trainX.shape,trainY.shape,spoken_length,written_length,spoken_vocab_size,written_vocab_size

((31728, 20), (31728, 20), 20, 20, 1376, 7723)

In [0]:
model = Sequential()
model.add(Embedding(spoken_vocab_size, 512, input_length=spoken_length, mask_zero=True))

model.add(LSTM(512))
model.add(RepeatVector(written_length))
model.add(LSTM(512, return_sequences=True))
model.add(Dense(written_vocab_size, activation='softmax'))

In [0]:
#https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/
rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [73]:
filename =dr+"letter_model.h5"
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1 ),  
          epochs=8, batch_size=32, 
          validation_split = 0.25,
          callbacks=[checkpoint], verbose=1)

Train on 23796 samples, validate on 7932 samples
Epoch 1/8
23796/23796 [==============================] - 626s 26ms/step - loss: 0.1544 - acc: 0.9781 - val_loss: 0.1624 - val_acc: 0.9773

Epoch 00001: val_loss improved from inf to 0.16238, saving model to /content/drive/My Drive/Classroom/Datasets/label_model.h5
Epoch 2/8
23796/23796 [==============================] - 623s 26ms/step - loss: 0.1423 - acc: 0.9793 - val_loss: 0.1566 - val_acc: 0.9788

Epoch 00002: val_loss improved from 0.16238 to 0.15656, saving model to /content/drive/My Drive/Classroom/Datasets/label_model.h5
Epoch 3/8
23796/23796 [==============================] - 630s 26ms/step - loss: 0.1329 - acc: 0.9799 - val_loss: 0.1510 - val_acc: 0.9796

Epoch 00003: val_loss improved from 0.15656 to 0.15096, saving model to /content/drive/My Drive/Classroom/Datasets/label_model.h5
Epoch 4/8
23796/23796 [==============================] - 620s 26ms/step - loss: 0.1267 - acc: 0.9808 - val_loss: 0.1504 - val_acc: 0.9791

Epoch 000

In [0]:
import pickle

# saving spoken token
with open(dr+'letter_spoken_tokenizer.pickle', 'wb') as handle:
    pickle.dump(spoken_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [0]:
# saving written token
with open(dr+'letter_written_tokenizer.pickle', 'wb') as handle:
    pickle.dump(written_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [0]:
# loading
with open(dr+'letter_spoken_tokenizer.pickle', 'rb') as handle:
    sp_t = pickle.load(handle)